# ĐỒ ÁN THỰC HÀNH
# CSC17104 – LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU

Giảng viên: Bùi Tiến Lên

Thông tin thành viên:
1. Nguyễn Văn Tuấn Đạt 19120472
2.
3. 
4. 

## Import

In [ ]:
import numpy as np
import pandas as pd

## A. Thu thập dữ liệu
- Tên dữ liệu: HR Analytics: Job Change of Data Scientists
- Nguồn lấy dữ liệu: kaggle
- Link dữ liệu: https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists
- License: CC0: Public Domain
- Mô tả về dữ liệu: dữ liệu gồm tập train và tập test. Trong đồ án này, nhóm sẽ sử dụng tập train.
            + Dữ liệu gồm 14 cột và 19158 dòng.

In [ ]:
df = pd.read_csv('aug_train.csv', encoding='utf-8')
df.shape

## B. Khám phá dữ liệu (thường đan xen với pha tiền xử lý dữ liệu)

### Mỗi dòng có ý nghĩa gì? Có vấn đề với các dòng có ý nghĩa khác nhau không?

- Theo mô tả dữ liệu của "HR Analytics: Job Change of Data Scientists": Mỗi dòng là thông tin tập huấn của mỗi thành viên trong công ty. Dữ liệu các dòng không bị trùng lặp.

In [ ]:
# YOUR CODE HERE
num_duplicated_rows = df.duplicated().sum()
have_duplicated_rows = True

if num_duplicated_rows == 0:
    have_duplicated_rows = False

have_duplicated_rows

### Mỗi cột có ý nghĩa gì?
            
            - enrollee_id : ID với từng ứng viên

            - city: Mã thành phố

            - city_ development _index : Chỉ số phát triển của thành phố (theo tỷ lệ)

            - gender: Giới tính

            - relevent_experience: Kinh nghiệm liên quan của ứng viên

            - enrolled_university: Loại khóa học đại học đã đăng ký nếu có

            - education_level: Trình độ học vấn

            - major_discipline: Ngành học chính

            - experience: Kinh nghiệm (tính theo năm)

            - company_size: Số lượng nhân viên trong công ty của người sử dụng lao động hiện tại

            - company_type : Loại chủ lao động hiện tại

            - lastnewjob: Sự khác biệt về số năm giữa công việc trước đây và công việc hiện tại

            - training_hours: Thời gian hoàn thành huấn luyện

            - target: 0 – Không nhảy việc, 1 – Nhảy việc

## Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

Dùng 'DataFrame.dtypes' để in ra kiểu dữ liệu chung của từng cột

In [ ]:
df.dtypes

In [ ]:
df['experience'].replace({'1':'1-5',
                                '2':'1-5',
                                '3':'1-5',
                                '4':'1-5',
                                '5':'1-5',
                                '6':'6-10',
                                '7':'6-10',
                                '8':'6-10',
                                '9':'6-10',
                                '10':'6-10',
                                '11':'11-15',
                                '12':'11-15',
                                '13':'11-15',
                                '14':'11-15',
                                '15':'11-15',
                                '16':'16-20',
                                '17':'16-20',
                                '18':'16-20',
                                '19':'16-20',
                                '20':'16-20'},inplace=True)

### Với mỗi cột, các giá trị được phân bố như thế nào?
##### Các cột có giá trị numeric ('city_development_index', 'training_hours', 'target')

In [ ]:
cols = ['city_development_index', 'training_hours', 'target']

missing_ratio = []
min = []
max = []
for col in cols:
    missing_ratio.append(df[col].isnull().sum() * 100 / len(df[col]))
    min.append(df[col].min())
    max.append(df[col].max())

values = [missing_ratio, min, max]
nume_col_profiles_df = pd.DataFrame(data=values, columns=cols)
nume_col_profiles_df.index = ['missing_ratio', 'min', 'max']

nume_col_profiles_df

##### Các cột có giá trị categorical

## C. Đưa ra các câu hỏi có ý nghĩa cần trả lời

## D. Tiền xử lý và phân tích dữ liệu để trả lời cho từng câu hỏi

## E. Tổng hợp lại quá trình thực hiện đồ án